In [0]:
!pip install lime
!pip install pandas
!pip install scikit-learn
!pip install pickle-mixin

In [0]:
import sklearn
import lime.lime_tabular
import pickle
import pandas as pd
import numpy as np

FEATURES_TO_OBSERVE = 10
filename = '/content/drive/My Drive/Predictions/Race_model.pickle'
rf_model = pickle.load(open(filename, 'rb'))

train_x = pd.read_pickle('/content/drive/My Drive/Predictions/Race_x_train.pickle')
train_y = pd.read_pickle('/content/drive/My Drive/Predictions/Race_y_train.pickle')
test_x = pd.read_pickle('/content/drive/My Drive/Predictions/Race_x_test.pickle')
test_y = pd.read_pickle('/content/drive/My Drive/Predictions/Race_y_test.pickle')

lst = rf_model.predict_proba(test_x.loc[[2957]])
print(lst)
print(np.array(rf_model.predict_proba(test_x.loc[[2957]])[2:3][0]))

for i in range(4):
  l = []
  predict_fn_rf = lambda x: np.array(rf_model.predict_proba(x)[i:i+1][0])
  #create explainer object
  explainer = lime.lime_tabular.LimeTabularExplainer(train_x.values,feature_names = train_x.columns,class_names=['0', '1'],kernel_width=5)

  c = 0
  indexes = train_x.index.values
  for j in indexes[0:1000]:
    print(c)
    choosen_instance = train_x.loc[[j]].values[0]
    exp = explainer.explain_instance(choosen_instance, predict_fn_rf,num_features=FEATURES_TO_OBSERVE)
    a = exp.as_list()
    l.append(a)
    c += 1

  pickle.dump( l, open( "/content/drive/My Drive/Predictions/predictionRace_" + str(i) + ".p", "wb" ) )


In [0]:
import pickle

filename = '/content/drive/My Drive/Predictions/predictionRace_0.p'
predictions_0 = pickle.load(open(filename, 'rb'))

filename = '/content/drive/My Drive/Predictions/predictionRace_1.p'
predictions_1 = pickle.load(open(filename, 'rb'))

filename = '/content/drive/My Drive/Predictions/predictionRace_2.p'
predictions_2 = pickle.load(open(filename, 'rb'))

filename = '/content/drive/My Drive/Predictions/predictionRace_3.p'
predictions_3 = pickle.load(open(filename, 'rb'))

def process_predictions(predictions):
  data_elements = []
  for i in range(len(predictions)):
    for j in range(len(predictions[i])):

      index = predictions[i][j][0].find('f')
      space = predictions[i][j][0][index:index+5].find(' ')
      data_elements.append(predictions[i][j][0][index:(index + space)])    

  return data_elements

pred_0  = process_predictions(predictions_0)
pred_1 = process_predictions(predictions_1)
pred_2 = process_predictions(predictions_2)
pred_3  = process_predictions(predictions_3)

In [0]:
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline

#https://stackoverflow.com/questions/3594514/how-to-find-most-common-elements-of-a-list/18587325

def visualize_results(results):
  x = []
  y = []
  most_common = [item for item in Counter(results).most_common()]
  for i in range(len(most_common)):
    x.append(most_common[i][0])
    y.append(most_common[i][1])

  plt.bar(x[0:10], y[0:10], color='red')
  plt.xlabel("Features")
  plt.ylabel("Count")
  plt.title("Feature distribution")
  #plt.show()
  #print('Most common features:', most_common[0][0],most_common[1][0],most_common[2][0])
  #print('Most common features:', most_common[0:5][0])
  common = []
  for i in range(10):
    common.append(most_common[i][0])

  print("top 10 feature:", common)

visualize_results(pred_0)
visualize_results(pred_1)
visualize_results(pred_2)
visualize_results(pred_3)